## 2. Data Engineering - Process CSV files into BQ Tables

### Create Spark session with BQ connector

Create a Spark session, connect to Hive Metastore and enable Hive support in Spark

In [1]:
# Run python kernel not pyspark kernel
# https://github.com/GoogleCloudDataproc/spark-bigquery-connector/blob/master/examples/notebooks/Top%20words%20in%20Shakespeare%20by%20work.ipynb
from pyspark.sql import SparkSession
from pyspark.sql.types import FloatType, IntegerType, StructField, StructType

from pyspark.sql import SparkSession
spark = SparkSession.builder \
.appName('Spark - Data Eng Demo') \
.config('spark.jars', 'gs://spark-lib/bigquery/spark-bigquery-latest.jar') \
.getOrCreate()

Check the first 1000 bytes of a file on GCS

In [2]:
!gsutil cat -h -r 0-1000 gs://datalake-vol2-data/dataset/fraud_data.csv

==> gs://datalake-vol2-data/dataset/fraud_data.csv <==
step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
1,TRANSFER,181.0,C1305486145,181.0,0.0,C553264065,0.0,0.0,1,0
1,CASH_OUT,181.0,C840083671,181.0,0.0,C38997010,21182.0,0.0,1,0
1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0
1,PAYMENT,7817.71,C90045638,53860.0,46042.29,M573487274,0.0,0.0,0,0
1,PAYMENT,7107.77,C154988899,183195.0,176087.23,M408069119,0.0,0.0,0,0
1,PAYMENT,7861.64,C1912850431,176087.23,168225.59,M633326333,0.0,0.0,0,0
1,PAYMENT,4024.36,C1265012928,2671.0,0.0,M1176932104,0.0,0.0,0,0
1,DEBIT,5337.77,C712410124,41720.0,36382.23,C195600860,41898.0,40348.79,0,0
1,DEBIT,9644.94,C1900366749,4465.0,0.0,C997608398,10845.0,157982.12,0,0
1,PAYMENT,3099.97,C249177573,20771.0,17671.03,M2096539129,0.0

In [3]:
path_to_train_csv = "gs://datalake-vol2-data/dataset/fraud_data.csv"

### Get Spark application ID 

This is useful to easily fine application in the Spark History UI

In [4]:
spark.conf.get("spark.app.id")

'application_1610100240292_0007'

Load the CSV file into a Spark Dataframe

In [5]:
df_bank_marketing_from_csv = spark \
.read \
.option("inferSchema" , "true") \
.option("header" , "true") \
.csv(path_to_train_csv)

In [7]:
df_bank_marketing_from_csv = df_bank_marketing_from_csv.drop("isFlaggedFraud")

In [8]:
df_bank_marketing_from_csv.printSchema()

root
 |-- step: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- nameOrig: string (nullable = true)
 |-- oldbalanceOrg: double (nullable = true)
 |-- newbalanceOrig: double (nullable = true)
 |-- nameDest: string (nullable = true)
 |-- oldbalanceDest: double (nullable = true)
 |-- newbalanceDest: double (nullable = true)
 |-- isFraud: integer (nullable = true)



In [10]:
# spark to bq datatypes -> https://github.com/GoogleCloudDataproc/spark-bigquery-connector#data-types
schema_inline = df_bank_marketing_from_csv.schema.simpleString().replace('struct<', '').replace('>', '').replace('int', 'int64').replace('double', 'numeric')
schema_inline

'step:int64,type:string,amount:numeric,nameOrig:string,oldbalanceOrg:numeric,newbalanceOrig:numeric,nameDest:string,oldbalanceDest:numeric,newbalanceDest:numeric,isFraud:int64'

In [11]:
df_bank_marketing_from_csv.show(5)

+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+
|step|    type|  amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|
+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+
|   1| PAYMENT| 9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|      0|
|   1| PAYMENT| 1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|      0|
|   1|TRANSFER|   181.0|C1305486145|        181.0|           0.0| C553264065|           0.0|           0.0|      1|
|   1|CASH_OUT|   181.0| C840083671|        181.0|           0.0|  C38997010|       21182.0|           0.0|      1|
|   1| PAYMENT|11668.14|C2048537720|      41554.0|      29885.86|M1230701703|           0.0|           0.0|      0|
+----+--------+--------+-----------+-------------+--------------+-------

Run transformations on the data

In [ ]:
## Any transformations on your data can be done at this point

In [12]:
# get name for dataset in BQ
project_id = !gcloud config list --format 'value(core.project)' 2>/dev/null 
dataset_raw_name = project_id[0] + '-raw'
dataset_raw_name = dataset_raw_name.replace('-', '_')
dataset_raw_name

'datalake_vol2_raw'

Create BQ dataset

In [ ]:
# !bq --location=europe-west3 mk -d \
# {dataset_raw_name}

In [15]:
# create path to new table for creation
bq_table_name = 'transaction_data'
bq_table_path= 'datalake_vol2_raw.' + bq_table_name
bq_table_path

'datalake_vol2_raw.transaction_data'

In [14]:
!bq mk --table \
{bq_table_path} \
{schema_inline}

Table 'datalake-vol2:datalake_vol2_raw.transaction_data' successfully created.


#### Check that table was created

In [23]:
# spark.sql("SHOW TABLES in bank_demo_db").show()
table = "datalake-vol2:" + bq_table_path
df_bank_marketing_from_bq_table = spark.read \
.format("bigquery") \
.option("table", table) \
.load()

In [17]:
df_bank_marketing_from_bq_table.printSchema()

root
 |-- step: long (nullable = true)
 |-- type: string (nullable = true)
 |-- amount: decimal(38,9) (nullable = true)
 |-- nameOrig: string (nullable = true)
 |-- oldbalanceOrg: decimal(38,9) (nullable = true)
 |-- newbalanceOrig: decimal(38,9) (nullable = true)
 |-- nameDest: string (nullable = true)
 |-- oldbalanceDest: decimal(38,9) (nullable = true)
 |-- newbalanceDest: decimal(38,9) (nullable = true)
 |-- isFraud: long (nullable = true)



In [18]:
df_bank_marketing_from_bq_table.show()

+----+----+------+--------+-------------+--------------+--------+--------------+--------------+-------+
|step|type|amount|nameOrig|oldbalanceOrg|newbalanceOrig|nameDest|oldbalanceDest|newbalanceDest|isFraud|
+----+----+------+--------+-------------+--------------+--------+--------------+--------------+-------+
+----+----+------+--------+-------------+--------------+--------+--------------+--------------+-------+



In [19]:
# create temp GCS bucket for writing spark df to bq table
gcs_bucket = project_id[0] + '-data'
gcs_bucket

'datalake-vol2-data'

In [20]:
df_bank_marketing_from_csv.write \
.format("bigquery") \
.option("table", table) \
.option("temporaryGcsBucket", gcs_bucket) \
.mode('overwrite') \
.save()

In [24]:
# read the data again 
df_bank_marketing_from_bq_table = spark.read \
.format("bigquery") \
.option("table", table) \
.load()

+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+-------+
|step|    type|   amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|
+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+-------+
|   1| PAYMENT|  9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|      0|
|   1| PAYMENT|  1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|      0|
|   1|TRANSFER|    181.0|C1305486145|        181.0|           0.0| C553264065|           0.0|           0.0|      1|
|   1|CASH_OUT|    181.0| C840083671|        181.0|           0.0|  C38997010|       21182.0|           0.0|      1|
|   1| PAYMENT| 11668.14|C2048537720|      41554.0|      29885.86|M1230701703|           0.0|           0.0|      0|
|   1| PAYMENT|  7817.71|  C90045638|      53860.0|      46042.2

In [ ]:
df_bank_marketing_from_bq_table.show()

In [22]:
%%bigquery
SELECT *
FROM `datalake-vol2.datalake_vol2_raw.transaction_data`
LIMIT 10

Downloading: 100%|██████████| 10/10 [00:00<00:00, 10.14rows/s]


,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.0,0.00,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.0,0.00,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.0,0.00,1
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.0,0.00,1
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.0,0.00,0
5,1,PAYMENT,7817.71,C90045638,53860.00,46042.29,M573487274,0.0,0.00,0
6,1,PAYMENT,7107.77,C154988899,183195.00,176087.23,M408069119,0.0,0.00,0
7,1,PAYMENT,7861.64,C1912850431,176087.23,168225.59,M633326333,0.0,0.00,0
8,1,PAYMENT,4024.36,C1265012928,2671.00,0.00,M1176932104,0.0,0.00,0
9,1,DEBIT,5337.77,C712410124,41720.00,36382.23,C195600860,41898.0,40348.79,0


### Compute statistics for columns in table

In [25]:
# spark.sql("DESCRIBE TABLE EXTENDED bank_demo_db.bank_marketing Age").show()
df_bank_marketing_from_bq_table.describe().show()

+-------+------------------+--------+------------------+-----------+------------------+------------------+-----------+------------------+------------------+--------------------+
|summary|              step|    type|            amount|   nameOrig|     oldbalanceOrg|    newbalanceOrig|   nameDest|    oldbalanceDest|    newbalanceDest|             isFraud|
+-------+------------------+--------+------------------+-----------+------------------+------------------+-----------+------------------+------------------+--------------------+
|  count|           6362620| 6362620|           6362620|    6362620|           6362620|           6362620|    6362620|           6362620|           6362620|             6362620|
|   mean|243.39724563151657|    null|179861.90354912292|       null| 833883.1040744851| 855113.6685785672|       null|1100701.6665196999|1224996.3982020712|0.001290820448180152|
| stddev| 142.3319710491244|    null| 603858.2314629285|       null|2888242.6730375285|2924048.5029542595|    